In [ ]:
%pip install datasets transformers --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 898.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import sklearn
import pandas as pd
import numpy as np
import random
import torch
import json

seed = 42

np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

Getting the dataset.

In [ ]:
from datasets import load_dataset
sem_eval_2018_task_1 = load_dataset('sem_eval_2018_task_1', 'subtask5.english', trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/6838 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3259 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/886 [00:00<?, ? examples/s]

#BERT

BERT + 100% gold standard data + synthetic data

NO ROLE

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

num_epochs = 5
batch_size = 32
learning_rate = 1e-5
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
mixed_df = pd.read_csv("full_gsd_multi_gpt_data.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_columns))
model.dropout = torch.nn.Dropout(dropout)  # Adding dropout to the model
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(mixed_df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(mixed_df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

# Evaluation on test set
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt").to(device)
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    outputs_test = model(input_ids=inputs_test['input_ids'], attention_mask=inputs_test['attention_mask'])
    logits_test = outputs_test.logits
    predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).cpu().numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test.cpu(), predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='micro') * 100

confusion_matrix_test = multilabel_confusion_matrix(labels_test.cpu(), predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.4667, Val Loss: 0.3740
Epoch 2, Train Loss: 0.3446, Val Loss: 0.3302
Epoch 3, Train Loss: 0.2990, Val Loss: 0.3174
Epoch 4, Train Loss: 0.2692, Val Loss: 0.3119
Epoch 5, Train Loss: 0.2463, Val Loss: 0.3096
Test Accuracy: 27.19
Test F1-macro: 52.46
Test F1-micro: 68.75
Confusion Matrix:
Label: anticipation
[[2757   77]
 [ 372   53]]

Label: optimism
[[1845  271]
 [ 338  805]]

Label: trust
[[3032   74]
 [ 143   10]]

Label: joy
[[1628  189]
 [ 260 1182]]

Label: love
[[2544  199]
 [ 204  312]]

Label: anger
[[2017  141]
 [ 317  784]]

Label: disgust
[[1942  218]
 [ 328  771]]

Label: pessimism
[[2778  106]
 [ 277   98]]

Label: sadness
[[2142  157]
 [ 386  574]]

Label: fear
[[2707   67]
 [ 171  314]]

Label: surprise
[[3085    4]
 [ 164    6]]



NO ROLE + NEWS ARTICLES

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

num_epochs = 5
batch_size = 32
learning_rate = 1e-5
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
mixed_df = pd.read_csv("full_gsd_multi_gpt_data_no_role_news.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_columns))
model.dropout = torch.nn.Dropout(dropout)  # Adding dropout to the model
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(mixed_df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(mixed_df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

# Evaluation on test set
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt").to(device)
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    outputs_test = model(input_ids=inputs_test['input_ids'], attention_mask=inputs_test['attention_mask'])
    logits_test = outputs_test.logits
    predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).cpu().numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test.cpu(), predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='micro') * 100

confusion_matrix_test = multilabel_confusion_matrix(labels_test.cpu(), predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.4861, Val Loss: 0.3784
Epoch 2, Train Loss: 0.3662, Val Loss: 0.3298
Epoch 3, Train Loss: 0.3154, Val Loss: 0.3168
Epoch 4, Train Loss: 0.2825, Val Loss: 0.3081
Epoch 5, Train Loss: 0.2575, Val Loss: 0.3114
Test Accuracy: 27.92
Test F1-macro: 53.48
Test F1-micro: 69.09
Confusion Matrix:
Label: anticipation
[[2743   91]
 [ 367   58]]

Label: optimism
[[1771  345]
 [ 255  888]]

Label: trust
[[3079   27]
 [ 144    9]]

Label: joy
[[1613  204]
 [ 246 1196]]

Label: love
[[2585  158]
 [ 232  284]]

Label: anger
[[2010  148]
 [ 332  769]]

Label: disgust
[[1952  208]
 [ 367  732]]

Label: pessimism
[[2754  130]
 [ 277   98]]

Label: sadness
[[2140  159]
 [ 369  591]]

Label: fear
[[2688   86]
 [ 146  339]]

Label: surprise
[[3077   12]
 [ 154   16]]



ROLE 1

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

num_epochs = 5
batch_size = 32
learning_rate = 1e-5
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
mixed_df = pd.read_csv("full_gsd_multi_gpt_data_role_1.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_columns))
model.dropout = torch.nn.Dropout(dropout)  # Adding dropout to the model
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(mixed_df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(mixed_df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

# Evaluation on test set
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt").to(device)
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    outputs_test = model(input_ids=inputs_test['input_ids'], attention_mask=inputs_test['attention_mask'])
    logits_test = outputs_test.logits
    predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).cpu().numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test.cpu(), predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='micro') * 100

confusion_matrix_test = multilabel_confusion_matrix(labels_test.cpu(), predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.4705, Val Loss: 0.3681
Epoch 2, Train Loss: 0.3430, Val Loss: 0.3397
Epoch 3, Train Loss: 0.2964, Val Loss: 0.3164
Epoch 4, Train Loss: 0.2671, Val Loss: 0.3069
Epoch 5, Train Loss: 0.2445, Val Loss: 0.3043
Test Accuracy: 28.35
Test F1-macro: 51.99
Test F1-micro: 69.41
Confusion Matrix:
Label: anticipation
[[2767   67]
 [ 373   52]]

Label: optimism
[[1837  279]
 [ 323  820]]

Label: trust
[[3083   23]
 [ 151    2]]

Label: joy
[[1646  171]
 [ 296 1146]]

Label: love
[[2633  110]
 [ 262  254]]

Label: anger
[[1986  172]
 [ 289  812]]

Label: disgust
[[1894  266]
 [ 278  821]]

Label: pessimism
[[2816   68]
 [ 300   75]]

Label: sadness
[[2124  175]
 [ 345  615]]

Label: fear
[[2702   72]
 [ 162  323]]

Label: surprise
[[3085    4]
 [ 160   10]]



ROLE 2

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

num_epochs = 5
batch_size = 32
learning_rate = 1e-5
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
mixed_df = pd.read_csv("full_gsd_multi_gpt_data_role_2.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_columns))
model.dropout = torch.nn.Dropout(dropout)  # Adding dropout to the model
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(mixed_df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(mixed_df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

# Evaluation on test set
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt").to(device)
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    outputs_test = model(input_ids=inputs_test['input_ids'], attention_mask=inputs_test['attention_mask'])
    logits_test = outputs_test.logits
    predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).cpu().numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test.cpu(), predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='micro') * 100

confusion_matrix_test = multilabel_confusion_matrix(labels_test.cpu(), predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.4884, Val Loss: 0.3749
Epoch 2, Train Loss: 0.3548, Val Loss: 0.3407
Epoch 3, Train Loss: 0.3037, Val Loss: 0.3202
Epoch 4, Train Loss: 0.2706, Val Loss: 0.3136
Epoch 5, Train Loss: 0.2477, Val Loss: 0.3118
Test Accuracy: 27.25
Test F1-macro: 52.59
Test F1-micro: 69.24
Confusion Matrix:
Label: anticipation
[[2753   81]
 [ 362   63]]

Label: optimism
[[1783  333]
 [ 261  882]]

Label: trust
[[3030   76]
 [ 138   15]]

Label: joy
[[1609  208]
 [ 244 1198]]

Label: love
[[2531  212]
 [ 192  324]]

Label: anger
[[1993  165]
 [ 300  801]]

Label: disgust
[[1934  226]
 [ 325  774]]

Label: pessimism
[[2806   78]
 [ 297   78]]

Label: sadness
[[2140  159]
 [ 389  571]]

Label: fear
[[2724   50]
 [ 185  300]]

Label: surprise
[[3087    2]
 [ 167    3]]



ROLE 3

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

num_epochs = 5
batch_size = 32
learning_rate = 1e-5
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
mixed_df = pd.read_csv("full_gsd_multi_gpt_data_role_3.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_columns))
model.dropout = torch.nn.Dropout(dropout)  # Adding dropout to the model
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(mixed_df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(mixed_df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

# Evaluation on test set
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt").to(device)
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    outputs_test = model(input_ids=inputs_test['input_ids'], attention_mask=inputs_test['attention_mask'])
    logits_test = outputs_test.logits
    predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).cpu().numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test.cpu(), predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='micro') * 100

confusion_matrix_test = multilabel_confusion_matrix(labels_test.cpu(), predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.4697, Val Loss: 0.3673
Epoch 2, Train Loss: 0.3402, Val Loss: 0.3298
Epoch 3, Train Loss: 0.2923, Val Loss: 0.3162
Epoch 4, Train Loss: 0.2634, Val Loss: 0.3137
Epoch 5, Train Loss: 0.2423, Val Loss: 0.3205
Test Accuracy: 27.16
Test F1-macro: 54.20
Test F1-micro: 69.56
Confusion Matrix:
Label: anticipation
[[2711  123]
 [ 353   72]]

Label: optimism
[[1811  305]
 [ 283  860]]

Label: trust
[[3050   56]
 [ 139   14]]

Label: joy
[[1673  144]
 [ 317 1125]]

Label: love
[[2576  167]
 [ 235  281]]

Label: anger
[[1937  221]
 [ 239  862]]

Label: disgust
[[1875  285]
 [ 277  822]]

Label: pessimism
[[2713  171]
 [ 234  141]]

Label: sadness
[[2107  192]
 [ 348  612]]

Label: fear
[[2679   95]
 [ 144  341]]

Label: surprise
[[3086    3]
 [ 164    6]]



ROLE 4

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

num_epochs = 5
batch_size = 32
learning_rate = 1e-5
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
mixed_df = pd.read_csv("full_gsd_multi_gpt_data_role_4.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_columns))
model.dropout = torch.nn.Dropout(dropout)  # Adding dropout to the model
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(mixed_df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(mixed_df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data (assuming df_val remains unchanged)
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

# Evaluation on test set (assuming df_test remains unchanged)
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt").to(device)
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    outputs_test = model(input_ids=inputs_test['input_ids'], attention_mask=inputs_test['attention_mask'])
    logits_test = outputs_test.logits
    predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).cpu().numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test.cpu(), predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='micro') * 100

confusion_matrix_test = multilabel_confusion_matrix(labels_test.cpu(), predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.4656, Val Loss: 0.3659
Epoch 2, Train Loss: 0.3482, Val Loss: 0.3304
Epoch 3, Train Loss: 0.3026, Val Loss: 0.3157
Epoch 4, Train Loss: 0.2741, Val Loss: 0.3099
Epoch 5, Train Loss: 0.2519, Val Loss: 0.3096
Test Accuracy: 28.44
Test F1-macro: 52.72
Test F1-micro: 69.38
Confusion Matrix:
Label: anticipation
[[2788   46]
 [ 389   36]]

Label: optimism
[[1766  350]
 [ 237  906]]

Label: trust
[[3086   20]
 [ 143   10]]

Label: joy
[[1598  219]
 [ 218 1224]]

Label: love
[[2656   87]
 [ 275  241]]

Label: anger
[[2006  152]
 [ 306  795]]

Label: disgust
[[1947  213]
 [ 345  754]]

Label: pessimism
[[2749  135]
 [ 266  109]]

Label: sadness
[[2157  142]
 [ 410  550]]

Label: fear
[[2702   72]
 [ 179  306]]

Label: surprise
[[3086    3]
 [ 157   13]]



ROLE 1 + NEWS ARTICLES

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

num_epochs = 5
batch_size = 32
learning_rate = 1e-5
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
mixed_df = pd.read_csv("full_gsd_multi_gpt_data_role_5.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_columns))
model.dropout = torch.nn.Dropout(dropout)  # Adding dropout to the model
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(mixed_df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(mixed_df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

# Evaluation on test set
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt").to(device)
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    outputs_test = model(input_ids=inputs_test['input_ids'], attention_mask=inputs_test['attention_mask'])
    logits_test = outputs_test.logits
    predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).cpu().numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test.cpu(), predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='micro') * 100

confusion_matrix_test = multilabel_confusion_matrix(labels_test.cpu(), predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.4957, Val Loss: 0.3754
Epoch 2, Train Loss: 0.3639, Val Loss: 0.3387
Epoch 3, Train Loss: 0.3161, Val Loss: 0.3237
Epoch 4, Train Loss: 0.2839, Val Loss: 0.3140
Epoch 5, Train Loss: 0.2587, Val Loss: 0.3135
Test Accuracy: 27.95
Test F1-macro: 53.04
Test F1-micro: 69.68
Confusion Matrix:
Label: anticipation
[[2774   60]
 [ 379   46]]

Label: optimism
[[1794  322]
 [ 274  869]]

Label: trust
[[3084   22]
 [ 144    9]]

Label: joy
[[1658  159]
 [ 300 1142]]

Label: love
[[2614  129]
 [ 235  281]]

Label: anger
[[1889  269]
 [ 220  881]]

Label: disgust
[[1815  345]
 [ 230  869]]

Label: pessimism
[[2750  134]
 [ 278   97]]

Label: sadness
[[2092  207]
 [ 347  613]]

Label: fear
[[2721   53]
 [ 184  301]]

Label: surprise
[[3085    4]
 [ 159   11]]



##Single Label BERT

No Role

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

num_epochs = 5
batch_size = 32
learning_rate = 1e-5
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
mixed_df = pd.read_csv("full_gsd_single_gpt_data_no_role.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_columns))
model.dropout = torch.nn.Dropout(dropout)  # Adding dropout to the model
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(mixed_df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(mixed_df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

# Evaluation on test set
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt").to(device)
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    outputs_test = model(input_ids=inputs_test['input_ids'], attention_mask=inputs_test['attention_mask'])
    logits_test = outputs_test.logits
    predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).cpu().numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test.cpu(), predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='micro') * 100

confusion_matrix_test = multilabel_confusion_matrix(labels_test.cpu(), predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.4462, Val Loss: 0.3742
Epoch 2, Train Loss: 0.3362, Val Loss: 0.3282
Epoch 3, Train Loss: 0.2897, Val Loss: 0.3136
Epoch 4, Train Loss: 0.2582, Val Loss: 0.3085
Epoch 5, Train Loss: 0.2341, Val Loss: 0.3124
Test Accuracy: 27.31
Test F1-macro: 50.85
Test F1-micro: 67.91
Confusion Matrix:
Label: anticipation
[[2768   66]
 [ 385   40]]

Label: optimism
[[1836  280]
 [ 378  765]]

Label: trust
[[3101    5]
 [ 152    1]]

Label: joy
[[1643  174]
 [ 284 1158]]

Label: love
[[2594  149]
 [ 239  277]]

Label: anger
[[2020  138]
 [ 328  773]]

Label: disgust
[[1958  202]
 [ 367  732]]

Label: pessimism
[[2832   52]
 [ 317   58]]

Label: sadness
[[2153  146]
 [ 399  561]]

Label: fear
[[2697   77]
 [ 156  329]]

Label: surprise
[[3085    4]
 [ 154   16]]



Role 1

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

num_epochs = 5
batch_size = 32
learning_rate = 1e-5
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
mixed_df = pd.read_csv("full_gsd_single_gpt_data_role_1.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_columns))
model.dropout = torch.nn.Dropout(dropout)  # Adding dropout to the model
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(mixed_df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(mixed_df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

# Evaluation on test set
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt").to(device)
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    outputs_test = model(input_ids=inputs_test['input_ids'], attention_mask=inputs_test['attention_mask'])
    logits_test = outputs_test.logits
    predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).cpu().numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test.cpu(), predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='micro') * 100

confusion_matrix_test = multilabel_confusion_matrix(labels_test.cpu(), predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.4815, Val Loss: 0.4016
Epoch 2, Train Loss: 0.3605, Val Loss: 0.3361
Epoch 3, Train Loss: 0.3028, Val Loss: 0.3176
Epoch 4, Train Loss: 0.2692, Val Loss: 0.3102
Epoch 5, Train Loss: 0.2438, Val Loss: 0.3031
Test Accuracy: 28.11
Test F1-macro: 51.73
Test F1-micro: 69.04
Confusion Matrix:
Label: anticipation
[[2791   43]
 [ 386   39]]

Label: optimism
[[1830  286]
 [ 369  774]]

Label: trust
[[3103    3]
 [ 153    0]]

Label: joy
[[1648  169]
 [ 293 1149]]

Label: love
[[2594  149]
 [ 231  285]]

Label: anger
[[1977  181]
 [ 271  830]]

Label: disgust
[[1906  254]
 [ 281  818]]

Label: pessimism
[[2807   77]
 [ 288   87]]

Label: sadness
[[2145  154]
 [ 397  563]]

Label: fear
[[2718   56]
 [ 169  316]]

Label: surprise
[[3086    3]
 [ 158   12]]



Role 1 + NEWS

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

num_epochs = 5
batch_size = 32
learning_rate = 1e-5
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
mixed_df = pd.read_csv("full_gsd_single_gpt_data_role_1_news.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_columns))
model.dropout = torch.nn.Dropout(dropout)  # Adding dropout to the model
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(mixed_df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(mixed_df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

# Evaluation on test set
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt").to(device)
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    outputs_test = model(input_ids=inputs_test['input_ids'], attention_mask=inputs_test['attention_mask'])
    logits_test = outputs_test.logits
    predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).cpu().numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test.cpu(), predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='micro') * 100

confusion_matrix_test = multilabel_confusion_matrix(labels_test.cpu(), predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.4528, Val Loss: 0.3726
Epoch 2, Train Loss: 0.3446, Val Loss: 0.3258
Epoch 3, Train Loss: 0.2958, Val Loss: 0.3093
Epoch 4, Train Loss: 0.2661, Val Loss: 0.3045
Epoch 5, Train Loss: 0.2429, Val Loss: 0.3076
Test Accuracy: 27.16
Test F1-macro: 53.01
Test F1-micro: 69.12
Confusion Matrix:
Label: anticipation
[[2751   83]
 [ 363   62]]

Label: optimism
[[1806  310]
 [ 335  808]]

Label: trust
[[3101    5]
 [ 148    5]]

Label: joy
[[1552  265]
 [ 210 1232]]

Label: love
[[2561  182]
 [ 211  305]]

Label: anger
[[2016  142]
 [ 318  783]]

Label: disgust
[[1941  219]
 [ 336  763]]

Label: pessimism
[[2781  103]
 [ 282   93]]

Label: sadness
[[2142  157]
 [ 375  585]]

Label: fear
[[2710   64]
 [ 168  317]]

Label: surprise
[[3085    4]
 [ 157   13]]



Increased multi-label

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

num_epochs = 5
batch_size = 32
learning_rate = 1e-5
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
mixed_df = pd.read_csv("full_gsd_increased_gpt_data_multi.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_columns))
model.dropout = torch.nn.Dropout(dropout)  # Adding dropout to the model
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(mixed_df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(mixed_df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

# Evaluation on test set
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt").to(device)
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    outputs_test = model(input_ids=inputs_test['input_ids'], attention_mask=inputs_test['attention_mask'])
    logits_test = outputs_test.logits
    predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).cpu().numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test.cpu(), predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='micro') * 100

confusion_matrix_test = multilabel_confusion_matrix(labels_test.cpu(), predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.4523, Val Loss: 0.3607
Epoch 2, Train Loss: 0.3128, Val Loss: 0.3285
Epoch 3, Train Loss: 0.2686, Val Loss: 0.3155
Epoch 4, Train Loss: 0.2400, Val Loss: 0.3122
Epoch 5, Train Loss: 0.2190, Val Loss: 0.3223
Test Accuracy: 26.70
Test F1-macro: 54.20
Test F1-micro: 69.71
Confusion Matrix:
Label: anticipation
[[2780   54]
 [ 373   52]]

Label: optimism
[[1824  292]
 [ 316  827]]

Label: trust
[[3036   70]
 [ 144    9]]

Label: joy
[[1674  143]
 [ 332 1110]]

Label: love
[[2543  200]
 [ 188  328]]

Label: anger
[[1966  192]
 [ 266  835]]

Label: disgust
[[1867  293]
 [ 266  833]]

Label: pessimism
[[2692  192]
 [ 236  139]]

Label: sadness
[[2013  286]
 [ 272  688]]

Label: fear
[[2693   81]
 [ 139  346]]

Label: surprise
[[3084    5]
 [ 160   10]]



Increased single-label

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

num_epochs = 5
batch_size = 32
learning_rate = 1e-5
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
mixed_df = pd.read_csv("full_gsd_increased_gpt_data_single.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_columns))
model.dropout = torch.nn.Dropout(dropout)  # Adding dropout to the model
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(mixed_df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(mixed_df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

# Evaluation on test set
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt").to(device)
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    outputs_test = model(input_ids=inputs_test['input_ids'], attention_mask=inputs_test['attention_mask'])
    logits_test = outputs_test.logits
    predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).cpu().numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test.cpu(), predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='micro') * 100

confusion_matrix_test = multilabel_confusion_matrix(labels_test.cpu(), predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.4230, Val Loss: 0.3613
Epoch 2, Train Loss: 0.3057, Val Loss: 0.3219
Epoch 3, Train Loss: 0.2549, Val Loss: 0.3136
Epoch 4, Train Loss: 0.2246, Val Loss: 0.3044
Epoch 5, Train Loss: 0.2015, Val Loss: 0.3136
Test Accuracy: 27.98
Test F1-macro: 53.01
Test F1-micro: 69.45
Confusion Matrix:
Label: anticipation
[[2788   46]
 [ 388   37]]

Label: optimism
[[1871  245]
 [ 406  737]]

Label: trust
[[3095   11]
 [ 146    7]]

Label: joy
[[1675  142]
 [ 323 1119]]

Label: love
[[2563  180]
 [ 220  296]]

Label: anger
[[1937  221]
 [ 234  867]]

Label: disgust
[[1830  330]
 [ 229  870]]

Label: pessimism
[[2806   78]
 [ 308   67]]

Label: sadness
[[2048  251]
 [ 282  678]]

Label: fear
[[2697   77]
 [ 153  332]]

Label: surprise
[[3079   10]
 [ 148   22]]



#RoBERTa

RoBERTa + GSD + synthetic data

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from transformers import AutoTokenizer, RobertaForSequenceClassification

# Define hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 1e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dropout = 0.2
gradient_accumulation_steps = 4

# Load dataset
df = pd.read_csv("full_gsd_multi_gpt_data.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model with dropout
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_columns), hidden_dropout_prob=dropout, attention_probs_dropout_prob=dropout)
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for step, batch in enumerate(train_dataloader):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        total_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Free up memory
        del input_ids, attention_mask, labels, outputs, loss
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            # Free up memory
            del input_ids, attention_mask, labels, outputs, loss
            torch.cuda.empty_cache()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

torch.cuda.empty_cache()

# Evaluation on test set in batches
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32)

test_data = TensorDataset(inputs_test['input_ids'], inputs_test['attention_mask'], labels_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

model.eval()
all_logits = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        all_logits.append(logits.cpu())

        # Free up memory
        del input_ids, attention_mask, labels, outputs, logits
        torch.cuda.empty_cache()

logits_test = torch.cat(all_logits, dim=0)
predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test, predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test, predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test, predicted_labels_test, average='micro') * 100
confusion_matrix_test = multilabel_confusion_matrix(labels_test, predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.5657, Val Loss: 0.4245
Epoch 2, Train Loss: 0.4122, Val Loss: 0.3629
Epoch 3, Train Loss: 0.3637, Val Loss: 0.3477
Epoch 4, Train Loss: 0.3375, Val Loss: 0.3279
Epoch 5, Train Loss: 0.3186, Val Loss: 0.3231
Epoch 6, Train Loss: 0.3037, Val Loss: 0.3152
Epoch 7, Train Loss: 0.2924, Val Loss: 0.3139
Epoch 8, Train Loss: 0.2820, Val Loss: 0.3113
Epoch 9, Train Loss: 0.2762, Val Loss: 0.3103
Epoch 10, Train Loss: 0.2683, Val Loss: 0.3020
Test Accuracy: 27.31
Test F1-macro: 53.76
Test F1-micro: 70.15
Confusion Matrix:
Label: anticipation
[[2768   66]
 [ 384   41]]

Label: optimism
[[1730  386]
 [ 223  920]]

Label: trust
[[3023   83]
 [ 145    8]]

Label: joy
[[1624  193]
 [ 256 1186]]

Label: love
[[2492  251]
 [ 156  360]]

Label: anger
[[1924  234]
 [ 222  879]]

Label: disgust
[[1881  279]
 [ 257  842]]

Label: pessimism
[[2725  159]
 [ 255  120]]

Label: sadness
[[2121  178]
 [ 353  607]]

Label: fear
[[2703   71]
 [ 173  312]]

Label: surprise
[[3070   19]
 [ 15

No role + NEWS

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer, RobertaForSequenceClassification

# Define hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 1e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dropout = 0.2
gradient_accumulation_steps = 4

# Load dataset
df = pd.read_csv("full_gsd_multi_gpt_data_no_role_news.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model with dropout
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_columns), hidden_dropout_prob=dropout, attention_probs_dropout_prob=dropout)
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for step, batch in enumerate(train_dataloader):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        total_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Free up memory
        del input_ids, attention_mask, labels, outputs, loss
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            # Free up memory
            del input_ids, attention_mask, labels, outputs, loss
            torch.cuda.empty_cache()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

torch.cuda.empty_cache()

# Evaluation on test set in batches
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32)

test_data = TensorDataset(inputs_test['input_ids'], inputs_test['attention_mask'], labels_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

model.eval()
all_logits = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        all_logits.append(logits.cpu())

        # Free up memory
        del input_ids, attention_mask, labels, outputs, logits
        torch.cuda.empty_cache()

logits_test = torch.cat(all_logits, dim=0)
predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test, predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test, predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test, predicted_labels_test, average='micro') * 100
confusion_matrix_test = multilabel_confusion_matrix(labels_test, predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.5690, Val Loss: 0.4298
Epoch 2, Train Loss: 0.4343, Val Loss: 0.3740
Epoch 3, Train Loss: 0.3846, Val Loss: 0.3545
Epoch 4, Train Loss: 0.3569, Val Loss: 0.3361
Epoch 5, Train Loss: 0.3377, Val Loss: 0.3309
Epoch 6, Train Loss: 0.3217, Val Loss: 0.3181
Epoch 7, Train Loss: 0.3098, Val Loss: 0.3159
Epoch 8, Train Loss: 0.2993, Val Loss: 0.3136
Epoch 9, Train Loss: 0.2906, Val Loss: 0.3077
Epoch 10, Train Loss: 0.2841, Val Loss: 0.3046
Test Accuracy: 28.29
Test F1-macro: 54.35
Test F1-micro: 70.75
Confusion Matrix:
Label: anticipation
[[2752   82]
 [ 371   54]]

Label: optimism
[[1713  403]
 [ 198  945]]

Label: trust
[[3065   41]
 [ 147    6]]

Label: joy
[[1600  217]
 [ 220 1222]]

Label: love
[[2474  269]
 [ 146  370]]

Label: anger
[[1914  244]
 [ 227  874]]

Label: disgust
[[1888  272]
 [ 250  849]]

Label: pessimism
[[2760  124]
 [ 280   95]]

Label: sadness
[[2108  191]
 [ 337  623]]

Label: fear
[[2709   65]
 [ 175  310]]

Label: surprise
[[3060   29]
 [ 15

ROLE 1

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from transformers import AutoTokenizer, RobertaForSequenceClassification

# Define hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 1e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dropout = 0.2
gradient_accumulation_steps = 4

# Load dataset
df = pd.read_csv("full_gsd_multi_gpt_data_role_1.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model with dropout
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_columns), hidden_dropout_prob=dropout, attention_probs_dropout_prob=dropout)
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for step, batch in enumerate(train_dataloader):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        total_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Free up memory
        del input_ids, attention_mask, labels, outputs, loss
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            # Free up memory
            del input_ids, attention_mask, labels, outputs, loss
            torch.cuda.empty_cache()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

torch.cuda.empty_cache()

# Evaluation on test set in batches
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32)

test_data = TensorDataset(inputs_test['input_ids'], inputs_test['attention_mask'], labels_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

model.eval()
all_logits = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        all_logits.append(logits.cpu())

        # Free up memory
        del input_ids, attention_mask, labels, outputs, logits
        torch.cuda.empty_cache()

logits_test = torch.cat(all_logits, dim=0)
predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test, predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test, predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test, predicted_labels_test, average='micro') * 100
confusion_matrix_test = multilabel_confusion_matrix(labels_test, predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.5655, Val Loss: 0.4474
Epoch 2, Train Loss: 0.4274, Val Loss: 0.3706
Epoch 3, Train Loss: 0.3707, Val Loss: 0.3549
Epoch 4, Train Loss: 0.3438, Val Loss: 0.3393
Epoch 5, Train Loss: 0.3234, Val Loss: 0.3239
Epoch 6, Train Loss: 0.3099, Val Loss: 0.3221
Epoch 7, Train Loss: 0.2962, Val Loss: 0.3124
Epoch 8, Train Loss: 0.2865, Val Loss: 0.3165
Epoch 9, Train Loss: 0.2784, Val Loss: 0.3086
Epoch 10, Train Loss: 0.2707, Val Loss: 0.3137
Test Accuracy: 26.30
Test F1-macro: 55.53
Test F1-micro: 70.09
Confusion Matrix:
Label: anticipation
[[2710  124]
 [ 351   74]]

Label: optimism
[[1655  461]
 [ 166  977]]

Label: trust
[[2971  135]
 [ 132   21]]

Label: joy
[[1594  223]
 [ 236 1206]]

Label: love
[[2442  301]
 [ 125  391]]

Label: anger
[[1982  176]
 [ 287  814]]

Label: disgust
[[1923  237]
 [ 291  808]]

Label: pessimism
[[2687  197]
 [ 229  146]]

Label: sadness
[[2109  190]
 [ 329  631]]

Label: fear
[[2684   90]
 [ 164  321]]

Label: surprise
[[3081    8]
 [ 15

ROLE 2

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer, RobertaForSequenceClassification

# Define hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 1e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dropout = 0.2
gradient_accumulation_steps = 4

# Load dataset
df = pd.read_csv("full_gsd_multi_gpt_data_role_2.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model with dropout
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_columns), hidden_dropout_prob=dropout, attention_probs_dropout_prob=dropout)
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for step, batch in enumerate(train_dataloader):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        total_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Free up memory
        del input_ids, attention_mask, labels, outputs, loss
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            # Free up memory
            del input_ids, attention_mask, labels, outputs, loss
            torch.cuda.empty_cache()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

torch.cuda.empty_cache()

# Evaluation on test set in batches
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32)

test_data = TensorDataset(inputs_test['input_ids'], inputs_test['attention_mask'], labels_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

model.eval()
all_logits = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        all_logits.append(logits.cpu())

        # Free up memory
        del input_ids, attention_mask, labels, outputs, logits
        torch.cuda.empty_cache()

logits_test = torch.cat(all_logits, dim=0)
predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test, predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test, predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test, predicted_labels_test, average='micro') * 100
confusion_matrix_test = multilabel_confusion_matrix(labels_test, predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.5761, Val Loss: 0.4410
Epoch 2, Train Loss: 0.4214, Val Loss: 0.3708
Epoch 3, Train Loss: 0.3684, Val Loss: 0.3463
Epoch 4, Train Loss: 0.3403, Val Loss: 0.3398
Epoch 5, Train Loss: 0.3206, Val Loss: 0.3233
Epoch 6, Train Loss: 0.3052, Val Loss: 0.3240
Epoch 7, Train Loss: 0.2964, Val Loss: 0.3141
Epoch 8, Train Loss: 0.2851, Val Loss: 0.3082
Epoch 9, Train Loss: 0.2758, Val Loss: 0.3164
Epoch 10, Train Loss: 0.2692, Val Loss: 0.3098
Test Accuracy: 26.88
Test F1-macro: 56.17
Test F1-micro: 70.00
Confusion Matrix:
Label: anticipation
[[2724  110]
 [ 354   71]]

Label: optimism
[[1786  330]
 [ 244  899]]

Label: trust
[[2951  155]
 [ 126   27]]

Label: joy
[[1669  148]
 [ 289 1153]]

Label: love
[[2525  218]
 [ 186  330]]

Label: anger
[[1950  208]
 [ 267  834]]

Label: disgust
[[1906  254]
 [ 268  831]]

Label: pessimism
[[2684  200]
 [ 236  139]]

Label: sadness
[[2098  201]
 [ 319  641]]

Label: fear
[[2662  112]
 [ 143  342]]

Label: surprise
[[3075   14]
 [ 15

ROLE 3

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from transformers import AutoTokenizer, RobertaForSequenceClassification

# Define hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 1e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dropout = 0.2
gradient_accumulation_steps = 4

# Load dataset
df = pd.read_csv("full_gsd_multi_gpt_data_role_3.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model with dropout
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_columns), hidden_dropout_prob=dropout, attention_probs_dropout_prob=dropout)
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for step, batch in enumerate(train_dataloader):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        total_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Free up memory
        del input_ids, attention_mask, labels, outputs, loss
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            # Free up memory
            del input_ids, attention_mask, labels, outputs, loss
            torch.cuda.empty_cache()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

torch.cuda.empty_cache()

# Evaluation on test set in batches
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32)

test_data = TensorDataset(inputs_test['input_ids'], inputs_test['attention_mask'], labels_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

model.eval()
all_logits = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        all_logits.append(logits.cpu())

        # Free up memory
        del input_ids, attention_mask, labels, outputs, logits
        torch.cuda.empty_cache()

logits_test = torch.cat(all_logits, dim=0)
predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test, predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test, predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test, predicted_labels_test, average='micro') * 100
confusion_matrix_test = multilabel_confusion_matrix(labels_test, predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.5581, Val Loss: 0.4243
Epoch 2, Train Loss: 0.4135, Val Loss: 0.3757
Epoch 3, Train Loss: 0.3680, Val Loss: 0.3614
Epoch 4, Train Loss: 0.3436, Val Loss: 0.3467
Epoch 5, Train Loss: 0.3237, Val Loss: 0.3342
Epoch 6, Train Loss: 0.3082, Val Loss: 0.3181
Epoch 7, Train Loss: 0.2967, Val Loss: 0.3215
Epoch 8, Train Loss: 0.2869, Val Loss: 0.3132
Epoch 9, Train Loss: 0.2796, Val Loss: 0.3108
Epoch 10, Train Loss: 0.2690, Val Loss: 0.3125
Test Accuracy: 26.60
Test F1-macro: 54.34
Test F1-micro: 69.86
Confusion Matrix:
Label: anticipation
[[2689  145]
 [ 351   74]]

Label: optimism
[[1757  359]
 [ 219  924]]

Label: trust
[[2976  130]
 [ 135   18]]

Label: joy
[[1647  170]
 [ 273 1169]]

Label: love
[[2474  269]
 [ 162  354]]

Label: anger
[[1937  221]
 [ 253  848]]

Label: disgust
[[1895  265]
 [ 272  827]]

Label: pessimism
[[2721  163]
 [ 257  118]]

Label: sadness
[[2138  161]
 [ 340  620]]

Label: fear
[[2672  102]
 [ 146  339]]

Label: surprise
[[3075   14]
 [ 16

ROLE 4

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer, RobertaForSequenceClassification

# Define hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 1e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dropout = 0.2
gradient_accumulation_steps = 4

# Load dataset
df = pd.read_csv("full_gsd_multi_gpt_data_role_4.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model with dropout
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_columns), hidden_dropout_prob=dropout, attention_probs_dropout_prob=dropout)
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for step, batch in enumerate(train_dataloader):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        total_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Free up memory
        del input_ids, attention_mask, labels, outputs, loss
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            # Free up memory
            del input_ids, attention_mask, labels, outputs, loss
            torch.cuda.empty_cache()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

torch.cuda.empty_cache()

# Evaluation on test set in batches
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32)

test_data = TensorDataset(inputs_test['input_ids'], inputs_test['attention_mask'], labels_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

model.eval()
all_logits = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        all_logits.append(logits.cpu())

        # Free up memory
        del input_ids, attention_mask, labels, outputs, logits
        torch.cuda.empty_cache()

logits_test = torch.cat(all_logits, dim=0)
predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test, predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test, predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test, predicted_labels_test, average='micro') * 100
confusion_matrix_test = multilabel_confusion_matrix(labels_test, predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.5584, Val Loss: 0.4515
Epoch 2, Train Loss: 0.4332, Val Loss: 0.3659
Epoch 3, Train Loss: 0.3801, Val Loss: 0.3439
Epoch 4, Train Loss: 0.3542, Val Loss: 0.3294
Epoch 5, Train Loss: 0.3315, Val Loss: 0.3209
Epoch 6, Train Loss: 0.3174, Val Loss: 0.3120
Epoch 7, Train Loss: 0.3062, Val Loss: 0.3085
Epoch 8, Train Loss: 0.2934, Val Loss: 0.3078
Epoch 9, Train Loss: 0.2846, Val Loss: 0.3081
Epoch 10, Train Loss: 0.2770, Val Loss: 0.2991
Test Accuracy: 29.21
Test F1-macro: 55.67
Test F1-micro: 70.42
Confusion Matrix:
Label: anticipation
[[2766   68]
 [ 367   58]]

Label: optimism
[[1790  326]
 [ 251  892]]

Label: trust
[[3070   36]
 [ 139   14]]

Label: joy
[[1595  222]
 [ 213 1229]]

Label: love
[[2602  141]
 [ 206  310]]

Label: anger
[[1995  163]
 [ 309  792]]

Label: disgust
[[1952  208]
 [ 320  779]]

Label: pessimism
[[2710  174]
 [ 247  128]]

Label: sadness
[[2138  161]
 [ 363  597]]

Label: fear
[[2697   77]
 [ 166  319]]

Label: surprise
[[3076   13]
 [ 14

ROLE 1 + NEWS

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from transformers import AutoTokenizer, RobertaForSequenceClassification

# Define hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 1e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dropout = 0.2
gradient_accumulation_steps = 4

# Load dataset
df = pd.read_csv("full_gsd_multi_gpt_data_role_5.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model with dropout
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_columns), hidden_dropout_prob=dropout, attention_probs_dropout_prob=dropout)
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for step, batch in enumerate(train_dataloader):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        total_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Free up memory
        del input_ids, attention_mask, labels, outputs, loss
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            # Free up memory
            del input_ids, attention_mask, labels, outputs, loss
            torch.cuda.empty_cache()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

torch.cuda.empty_cache()

# Evaluation on test set in batches
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32)

test_data = TensorDataset(inputs_test['input_ids'], inputs_test['attention_mask'], labels_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

model.eval()
all_logits = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        all_logits.append(logits.cpu())

        # Free up memory
        del input_ids, attention_mask, labels, outputs, logits
        torch.cuda.empty_cache()

logits_test = torch.cat(all_logits, dim=0)
predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test, predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test, predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test, predicted_labels_test, average='micro') * 100
confusion_matrix_test = multilabel_confusion_matrix(labels_test, predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.5687, Val Loss: 0.4626
Epoch 2, Train Loss: 0.4480, Val Loss: 0.3765
Epoch 3, Train Loss: 0.3861, Val Loss: 0.3555
Epoch 4, Train Loss: 0.3584, Val Loss: 0.3398
Epoch 5, Train Loss: 0.3383, Val Loss: 0.3345
Epoch 6, Train Loss: 0.3222, Val Loss: 0.3233
Epoch 7, Train Loss: 0.3100, Val Loss: 0.3221
Epoch 8, Train Loss: 0.3013, Val Loss: 0.3098
Epoch 9, Train Loss: 0.2917, Val Loss: 0.3062
Epoch 10, Train Loss: 0.2847, Val Loss: 0.3116
Test Accuracy: 27.62
Test F1-macro: 55.96
Test F1-micro: 70.03
Confusion Matrix:
Label: anticipation
[[2668  166]
 [ 339   86]]

Label: optimism
[[1739  377]
 [ 207  936]]

Label: trust
[[3050   56]
 [ 142   11]]

Label: joy
[[1632  185]
 [ 269 1173]]

Label: love
[[2551  192]
 [ 172  344]]

Label: anger
[[1969  189]
 [ 287  814]]

Label: disgust
[[1936  224]
 [ 301  798]]

Label: pessimism
[[2709  175]
 [ 249  126]]

Label: sadness
[[2120  179]
 [ 336  624]]

Label: fear
[[2601  173]
 [ 118  367]]

Label: surprise
[[3022   67]
 [ 14

##RoBERTa Single Label

NO ROLE

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from transformers import AutoTokenizer, RobertaForSequenceClassification

# Define hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 1e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dropout = 0.2
gradient_accumulation_steps = 4

# Load dataset
df = pd.read_csv("full_gsd_single_gpt_data_no_role.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model with dropout
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_columns), hidden_dropout_prob=dropout, attention_probs_dropout_prob=dropout)
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for step, batch in enumerate(train_dataloader):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        total_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Free up memory
        del input_ids, attention_mask, labels, outputs, loss
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            # Free up memory
            del input_ids, attention_mask, labels, outputs, loss
            torch.cuda.empty_cache()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

torch.cuda.empty_cache()

# Evaluation on test set in batches
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32)

test_data = TensorDataset(inputs_test['input_ids'], inputs_test['attention_mask'], labels_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

model.eval()
all_logits = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        all_logits.append(logits.cpu())

        # Free up memory
        del input_ids, attention_mask, labels, outputs, logits
        torch.cuda.empty_cache()

logits_test = torch.cat(all_logits, dim=0)
predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test, predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test, predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test, predicted_labels_test, average='micro') * 100
confusion_matrix_test = multilabel_confusion_matrix(labels_test, predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.5293, Val Loss: 0.4714
Epoch 2, Train Loss: 0.4160, Val Loss: 0.3738
Epoch 3, Train Loss: 0.3699, Val Loss: 0.3441
Epoch 4, Train Loss: 0.3386, Val Loss: 0.3301
Epoch 5, Train Loss: 0.3145, Val Loss: 0.3187
Epoch 6, Train Loss: 0.2976, Val Loss: 0.3118
Epoch 7, Train Loss: 0.2861, Val Loss: 0.3021
Epoch 8, Train Loss: 0.2736, Val Loss: 0.3038
Epoch 9, Train Loss: 0.2653, Val Loss: 0.3014
Epoch 10, Train Loss: 0.2588, Val Loss: 0.2980
Test Accuracy: 27.95
Test F1-macro: 52.39
Test F1-micro: 69.43
Confusion Matrix:
Label: anticipation
[[2773   61]
 [ 374   51]]

Label: optimism
[[1818  298]
 [ 359  784]]

Label: trust
[[3096   10]
 [ 146    7]]

Label: joy
[[1670  147]
 [ 317 1125]]

Label: love
[[2607  136]
 [ 231  285]]

Label: anger
[[1928  230]
 [ 231  870]]

Label: disgust
[[1893  267]
 [ 272  827]]

Label: pessimism
[[2847   37]
 [ 332   43]]

Label: sadness
[[2116  183]
 [ 328  632]]

Label: fear
[[2705   69]
 [ 174  311]]

Label: surprise
[[3081    8]
 [ 15

ROLE 1

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from transformers import AutoTokenizer, RobertaForSequenceClassification

# Define hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 1e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dropout = 0.2
gradient_accumulation_steps = 4

# Load dataset
df = pd.read_csv("full_gsd_single_gpt_data_role_1.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model with dropout
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_columns), hidden_dropout_prob=dropout, attention_probs_dropout_prob=dropout)
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for step, batch in enumerate(train_dataloader):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        total_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Free up memory
        del input_ids, attention_mask, labels, outputs, loss
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            # Free up memory
            del input_ids, attention_mask, labels, outputs, loss
            torch.cuda.empty_cache()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

torch.cuda.empty_cache()

# Evaluation on test set in batches
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32)

test_data = TensorDataset(inputs_test['input_ids'], inputs_test['attention_mask'], labels_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

model.eval()
all_logits = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        all_logits.append(logits.cpu())

        # Free up memory
        del input_ids, attention_mask, labels, outputs, logits
        torch.cuda.empty_cache()

logits_test = torch.cat(all_logits, dim=0)
predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test, predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test, predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test, predicted_labels_test, average='micro') * 100
confusion_matrix_test = multilabel_confusion_matrix(labels_test, predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.5332, Val Loss: 0.4709
Epoch 2, Train Loss: 0.4170, Val Loss: 0.3720
Epoch 3, Train Loss: 0.3686, Val Loss: 0.3483
Epoch 4, Train Loss: 0.3426, Val Loss: 0.3383
Epoch 5, Train Loss: 0.3190, Val Loss: 0.3211
Epoch 6, Train Loss: 0.3034, Val Loss: 0.3118
Epoch 7, Train Loss: 0.2895, Val Loss: 0.3079
Epoch 8, Train Loss: 0.2786, Val Loss: 0.3034
Epoch 9, Train Loss: 0.2698, Val Loss: 0.3021
Epoch 10, Train Loss: 0.2617, Val Loss: 0.2978
Test Accuracy: 29.36
Test F1-macro: 53.25
Test F1-micro: 70.48
Confusion Matrix:
Label: anticipation
[[2800   34]
 [ 388   37]]

Label: optimism
[[1753  363]
 [ 275  868]]

Label: trust
[[3099    7]
 [ 151    2]]

Label: joy
[[1604  213]
 [ 238 1204]]

Label: love
[[2531  212]
 [ 180  336]]

Label: anger
[[1944  214]
 [ 234  867]]

Label: disgust
[[1906  254]
 [ 277  822]]

Label: pessimism
[[2812   72]
 [ 292   83]]

Label: sadness
[[2133  166]
 [ 348  612]]

Label: fear
[[2730   44]
 [ 191  294]]

Label: surprise
[[3084    5]
 [ 15

ROLE 1 + NEWS

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from transformers import AutoTokenizer, RobertaForSequenceClassification

# Define hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 1e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dropout = 0.2
gradient_accumulation_steps = 4

# Load dataset
df = pd.read_csv("full_gsd_single_gpt_data_role_1_news.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model with dropout
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_columns), hidden_dropout_prob=dropout, attention_probs_dropout_prob=dropout)
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for step, batch in enumerate(train_dataloader):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        total_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Free up memory
        del input_ids, attention_mask, labels, outputs, loss
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            # Free up memory
            del input_ids, attention_mask, labels, outputs, loss
            torch.cuda.empty_cache()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

torch.cuda.empty_cache()

# Evaluation on test set in batches
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32)

test_data = TensorDataset(inputs_test['input_ids'], inputs_test['attention_mask'], labels_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

model.eval()
all_logits = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        all_logits.append(logits.cpu())

        # Free up memory
        del input_ids, attention_mask, labels, outputs, logits
        torch.cuda.empty_cache()

logits_test = torch.cat(all_logits, dim=0)
predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test, predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test, predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test, predicted_labels_test, average='micro') * 100
confusion_matrix_test = multilabel_confusion_matrix(labels_test, predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.5343, Val Loss: 0.4763
Epoch 2, Train Loss: 0.4328, Val Loss: 0.3787
Epoch 3, Train Loss: 0.3748, Val Loss: 0.3438
Epoch 4, Train Loss: 0.3445, Val Loss: 0.3283
Epoch 5, Train Loss: 0.3213, Val Loss: 0.3191
Epoch 6, Train Loss: 0.3059, Val Loss: 0.3115
Epoch 7, Train Loss: 0.2945, Val Loss: 0.3060
Epoch 8, Train Loss: 0.2837, Val Loss: 0.3063
Epoch 9, Train Loss: 0.2748, Val Loss: 0.3021
Epoch 10, Train Loss: 0.2689, Val Loss: 0.2988
Test Accuracy: 28.81
Test F1-macro: 52.70
Test F1-micro: 70.12
Confusion Matrix:
Label: anticipation
[[2800   34]
 [ 391   34]]

Label: optimism
[[1735  381]
 [ 254  889]]

Label: trust
[[3106    0]
 [ 152    1]]

Label: joy
[[1645  172]
 [ 263 1179]]

Label: love
[[2530  213]
 [ 163  353]]

Label: anger
[[1977  181]
 [ 278  823]]

Label: disgust
[[1934  226]
 [ 317  782]]

Label: pessimism
[[2812   72]
 [ 294   81]]

Label: sadness
[[2128  171]
 [ 347  613]]

Label: fear
[[2710   64]
 [ 187  298]]

Label: surprise
[[3082    7]
 [ 15

Increased multi-label

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from transformers import AutoTokenizer, RobertaForSequenceClassification

# Define hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 1e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dropout = 0.2
gradient_accumulation_steps = 4

# Load dataset
df = pd.read_csv("full_gsd_increased_gpt_data_multi.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model with dropout
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_columns), hidden_dropout_prob=dropout, attention_probs_dropout_prob=dropout)
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for step, batch in enumerate(train_dataloader):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        total_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Free up memory
        del input_ids, attention_mask, labels, outputs, loss
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            # Free up memory
            del input_ids, attention_mask, labels, outputs, loss
            torch.cuda.empty_cache()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

torch.cuda.empty_cache()

# Evaluation on test set in batches
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32)

test_data = TensorDataset(inputs_test['input_ids'], inputs_test['attention_mask'], labels_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

model.eval()
all_logits = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        all_logits.append(logits.cpu())

        # Free up memory
        del input_ids, attention_mask, labels, outputs, logits
        torch.cuda.empty_cache()

logits_test = torch.cat(all_logits, dim=0)
predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test, predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test, predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test, predicted_labels_test, average='micro') * 100
confusion_matrix_test = multilabel_confusion_matrix(labels_test, predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.6135, Val Loss: 0.4731
Epoch 2, Train Loss: 0.4339, Val Loss: 0.3949
Epoch 3, Train Loss: 0.3595, Val Loss: 0.3745
Epoch 4, Train Loss: 0.3273, Val Loss: 0.3593
Epoch 5, Train Loss: 0.3043, Val Loss: 0.3479
Epoch 6, Train Loss: 0.2888, Val Loss: 0.3444
Epoch 7, Train Loss: 0.2716, Val Loss: 0.3332
Epoch 8, Train Loss: 0.2606, Val Loss: 0.3364
Epoch 9, Train Loss: 0.2525, Val Loss: 0.3330
Epoch 10, Train Loss: 0.2407, Val Loss: 0.3204
Test Accuracy: 24.42
Test F1-macro: 52.58
Test F1-micro: 67.58
Confusion Matrix:
Label: anticipation
[[2698  136]
 [ 376   49]]

Label: optimism
[[1770  346]
 [ 279  864]]

Label: trust
[[2949  157]
 [ 135   18]]

Label: joy
[[1617  200]
 [ 283 1159]]

Label: love
[[2558  185]
 [ 213  303]]

Label: anger
[[1941  217]
 [ 268  833]]

Label: disgust
[[1907  253]
 [ 285  814]]

Label: pessimism
[[2694  190]
 [ 253  122]]

Label: sadness
[[2049  250]
 [ 353  607]]

Label: fear
[[2659  115]
 [ 186  299]]

Label: surprise
[[3048   41]
 [ 15

Increased single-label

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from transformers import AutoTokenizer, RobertaForSequenceClassification

# Define hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 1e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dropout = 0.2
gradient_accumulation_steps = 4

# Load dataset (assuming combined_df and sem_eval_2018_task_1 are defined)
df = pd.read_csv("full_gsd_increased_gpt_data_single.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model with dropout
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_columns), hidden_dropout_prob=dropout, attention_probs_dropout_prob=dropout)
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for step, batch in enumerate(train_dataloader):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        total_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Free up memory
        del input_ids, attention_mask, labels, outputs, loss
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            # Free up memory
            del input_ids, attention_mask, labels, outputs, loss
            torch.cuda.empty_cache()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

torch.cuda.empty_cache()

# Evaluation on test set in batches
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32)

test_data = TensorDataset(inputs_test['input_ids'], inputs_test['attention_mask'], labels_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

model.eval()
all_logits = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        all_logits.append(logits.cpu())

        # Free up memory
        del input_ids, attention_mask, labels, outputs, logits
        torch.cuda.empty_cache()

logits_test = torch.cat(all_logits, dim=0)
predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test, predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test, predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test, predicted_labels_test, average='micro') * 100
confusion_matrix_test = multilabel_confusion_matrix(labels_test, predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.5494, Val Loss: 0.4948
Epoch 2, Train Loss: 0.4178, Val Loss: 0.4276
Epoch 3, Train Loss: 0.3669, Val Loss: 0.3920
Epoch 4, Train Loss: 0.3266, Val Loss: 0.3632
Epoch 5, Train Loss: 0.2968, Val Loss: 0.3502
Epoch 6, Train Loss: 0.2746, Val Loss: 0.3341
Epoch 7, Train Loss: 0.2524, Val Loss: 0.3270
Epoch 8, Train Loss: 0.2405, Val Loss: 0.3238
Epoch 9, Train Loss: 0.2267, Val Loss: 0.3175
Epoch 10, Train Loss: 0.2176, Val Loss: 0.3152
Test Accuracy: 25.50
Test F1-macro: 49.97
Test F1-micro: 67.15
Confusion Matrix:
Label: anticipation
[[2787   47]
 [ 382   43]]

Label: optimism
[[1850  266]
 [ 408  735]]

Label: trust
[[3090   16]
 [ 145    8]]

Label: joy
[[1668  149]
 [ 340 1102]]

Label: love
[[2604  139]
 [ 257  259]]

Label: anger
[[1937  221]
 [ 273  828]]

Label: disgust
[[1885  275]
 [ 273  826]]

Label: pessimism
[[2833   51]
 [ 340   35]]

Label: sadness
[[2118  181]
 [ 393  567]]

Label: fear
[[2660  114]
 [ 183  302]]

Label: surprise
[[3076   13]
 [ 15